In [120]:
# 库文件导入 
import pyrealsense2 as rs  #D435I库
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import time

In [121]:
# 配置深度和彩色数据流
start = time.time()              #记录程序运行起始时间
pipeline = rs.pipeline()         #定义通道变量，简化缩写
config = rs.config()             #数据流配置简写，允许管道用户为管道流以及设备选择和配置请求过滤器
#config.enable_all_streams()     #显示启用所有设备流
config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)   #显示启用深度流，30是帧率
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)  #显示启用彩色流
config.enable_stream(rs.stream.infrared, 1, 1280, 720, rs.format.y8, 30) #显示启用红外流1
config.enable_stream(rs.stream.infrared, 2, 1280, 720, rs.format.y8, 30) #显示启用红外流1
#开始采集
profile = pipeline.start(config)
cv2.waitKey(1000)     # 等待时间(ms) = 获取图像，realsense刚启动的时候图像会有一些失真
# 深度图像向彩色对齐
align_to_color = rs.align(rs.stream.color)#简化缩写

In [133]:
#===获得文件最大序号值===#
def Get_IMG_SeqNum(root = './', img_suffix = '.png', img_name_start = 'color', img_name_startnum = 5): 
    # root = './'  #指定目录 
    Seq_num_Png = [] 
    for files in os.listdir(root): 
        if (os.path.splitext(files)[1]== img_suffix) & (os.path.splitext(files)[0][:img_name_startnum]==img_name_start):  
            Seq_num_Png.append(int(os.path.splitext(files)[0][-6:])) 
    if len(Seq_num_Png)==0:  #列表为空 返回0  
        Seq_num_Png.append(0) 
    return max(Seq_num_Png)  

In [123]:
#===深度像素单位 与 分割距离定义===#
# Getting the depth sensor's depth scale (see rs-align example for explanation)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)

# We will be removing the background of objects more than
# clipping_distance_in_meters meters away
clipping_distance_in_meters = 1       #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale

Depth Scale is:  0.0010000000474974513


In [124]:
def Collect_Data():   
    # 等待连贯的帧:深度和颜色
    frames = pipeline.wait_for_frames()
    # 在上面稳定的帧上运行对齐算法以获得一组对齐的图像  对齐是指RGB+D
    frames = align_to_color.process(frames)
    # 从上面的对齐图像中抽出深度图像和彩色图像
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()
    # 得到红外图像
    ir_left = frames.get_infrared_frame(1)
    ir_right = frames.get_infrared_frame(2)
    #判断是否两个图像是否有图像 
    if not depth_frame or not color_frame:
        print("NO depth_frame or NO color_frame") 
    #将D图像转换为伪彩色图像       
    # 将RGB与D图像转换为numpy数组  
    depth_image = np.asanyarray(depth_frame.get_data()) 
    color_image = np.asanyarray(color_frame.get_data()) 

    #将红外图像转换为numpy数组
    ir_left_image = np.asanyarray(ir_left.get_data())   
    ir_right_image = np.asanyarray(ir_right.get_data()) 

    # 在深度图像上应用伪彩色图像算法(图像必须通过cv2.convertScaleAbs转换为每像素8位) 
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET) 

    #===图像显示===#
    # 将两幅图像举证就行行连接 
    images = np.hstack((color_image, depth_colormap)) 
    # 合成的图像进行显示 
    cv2.namedWindow('Align Example', cv2.WINDOW_NORMAL)  # 窗口大小保持比例
    cv2.imshow('Align Example', images) 
    key = cv2.waitKey(1) # 等待时间(ms)或按键 

    #===图像保存===#
    #时间记录   #固定序号
    localtime =time.localtime(time.time()) 
    tname = time.strftime("%Y%m%d_%H%M%S", time.localtime()) 

    #图像路径
    Image_path = ["./Image_NPY/NPY_color_depth/", "./Image_NPY/NPY_Infrared/",  
                "./Image_IMG/IMG_color_depth/", "./Image_IMG/IMG_Infrared/"] 
    # 图片计数序号
    Fig_CNT_Seq = Get_IMG_SeqNum(root=Image_path[2]) + 1  #color iamge path
    str_Fig_CNT_Seq = '_' + str(Fig_CNT_Seq).zfill(6)   

    #保存图像npy数据 color_image_npy + depth_image_npy 
    np.save(file = Image_path[0] +'depth'+ str_Fig_CNT_Seq +'.npy', arr = depth_image) 
    np.save(file = Image_path[0] +'color'+ str_Fig_CNT_Seq +'.npy', arr = color_image) 
    #保存图像npy数据 ir_left_npy + ir_right_npy 
    np.save(file = Image_path[1] +'ir_left'+ str_Fig_CNT_Seq +'.npy', arr = ir_left_image) 
    np.save(file = Image_path[1] +'ir_right'+ str_Fig_CNT_Seq +'.npy', arr = ir_right_image) 

    #保存图像color_image + depth_image 
    cv2.imwrite( Image_path[2] + 'color' + str_Fig_CNT_Seq +'.png', color_image) 
    cv2.imwrite( Image_path[2] + 'depth' + str_Fig_CNT_Seq +'.png', depth_image) 
    cv2.imwrite( Image_path[2] + 'depth_cMAP' + str_Fig_CNT_Seq +'.png', depth_colormap) 
    #保存图像 infrared image 
    cv2.imwrite( Image_path[3] + 'ir_left' + str_Fig_CNT_Seq + '.png', ir_left_image)
    cv2.imwrite( Image_path[3] + 'ir_right' + str_Fig_CNT_Seq + '.png', ir_right_image)



In [164]:
#===检查文件序号值===#
# 定义要创建的目录
Image_path= ["./Image_NPY/NPY_color_depth/", "./Image_NPY/NPY_Infrared/",  
         "./Image_IMG/IMG_color_depth/", "./Image_IMG/IMG_Infrared/"] 

def Check_IMG_SeqNum(root = './', img_suffix = '.npy', img_name_start =['color','depth'], img_name_startnum = 5):
    # root = './'  #指定目录
    Seq_num_Color = []
    Seq_num_Depth = []
    for files in os.listdir(root):
        if (os.path.splitext(files)[1]== img_suffix) & (os.path.splitext(files)[0][:img_name_startnum]==img_name_start[0]):  
            Seq_num_Color.append(int(os.path.splitext(files)[0][-6:]))

        if (os.path.splitext(files)[1]== img_suffix) & (os.path.splitext(files)[0][:img_name_startnum]==img_name_start[1]):  
            Seq_num_Depth.append(int(os.path.splitext(files)[0][-6:]))
    
    if len(Seq_num_Color)==0:     #列表为空  返回0  
        Seq_num_Color.append(0)     
    if len(Seq_num_Depth)==0:     #列表为空  返回0  
        Seq_num_Depth.append(0)   
    if np.equal(Seq_num_Color, Seq_num_Depth).all: 
        print("Check_IMG_SeqNum OK") 
    else:
        print("Warnning: Check_IMG_SeqNum")

    IMGNPY_dict = {'color_npy_num':np.array(Seq_num_Color), 'depth_npy_num':np.array(Seq_num_Depth)} #np.array（显示比较好）与 list 转换
    return IMGNPY_dict  
#执行=检查文件序号值   
Check_IMG_SeqNum(root=Image_path[0]) 

Check_IMG_SeqNum OK


{'color_npy_num': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20]),
 'depth_npy_num': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20])}

In [174]:
#===执行采集函数===# 
Collect_Data() 
#执行=检查文件序号值    
Check_IMG_SeqNum(root=Image_path[0]) 

Check_IMG_SeqNum OK


{'color_npy_num': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]),
 'depth_npy_num': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])}

In [175]:
#===关闭摄像头 与 窗口===#
Flag_OFFCamer = 1  
if Flag_OFFCamer:
    cv2.destroyAllWindows()
    pipeline.stop()


In [117]:
#===独立数据 = 处理环节===#    
depth_scale = 0.0010000000474974513 
clipping_distance_in_meters = 1       #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale

Seq_num = 1
#图像路径
Image_path= ["./Image_NPY/NPY_color_depth/", "./Image_NPY/NPY_Infrared/",  
         "./Image_IMG/IMG_color_depth/", "./Image_IMG/IMG_Infrared/"] 

color_image_npy = np.load(file = Image_path[0] +'color_'+ str(Seq_num).zfill(6) +'.npy')
deep_image_npy = np.load(file = Image_path[0] +'depth_'+ str(Seq_num).zfill(6) +'.npy')
deep_image_npy_cMAP = cv2.applyColorMap(cv2.convertScaleAbs(deep_image_npy, alpha=0.03), cv2.COLORMAP_JET)

deep_image_npy_3d = np.dstack((deep_image_npy, deep_image_npy, deep_image_npy))
gray_color = np.ones_like(color_image_npy) * 153
backg_remove = np.where( (deep_image_npy_3d > clipping_distance) | (deep_image_npy_3d < 0) , gray_color , color_image_npy )


images = np.hstack((backg_remove, deep_image_npy_cMAP))


cv2.namedWindow('Align Example', cv2.WINDOW_KEEPRATIO)    # 窗口大小保持比例
cv2.imshow('Align Example', images)


# Press esc or 'q' to close the image window
while True:
    key = cv2.waitKey(1)
    if key & 0xFF == ord('q') or key == 27:
        cv2.destroyAllWindows()
        break

In [137]:
#===创建的目录= 函数===#
def mkdir(path):
    import os      # 引入模块
    isExists=os.path.exists(path) # 判断路径是否存在 #存在True  #不存在 False
    if not isExists: # 判断结果
        os.makedirs(path) # 创建目录操作函数
        print (path +' 创建成功')
        return True
    else:  
        print (path+' 目录已存在')# 如果目录存在则不创建，并提示目录已存在
        return False
        
# 定义要创建的目录
Image_path= ["./Image_NPY/NPY_color_depth/", "./Image_NPY/NPY_Infrared/",  
         "./Image_IMG/IMG_color_depth/", "./Image_IMG/IMG_Infrared/"] 
# 调用函数
for path in Image_path:
    mkdir(path)



./Image_NPY/NPY_color_depth/ 创建成功
./Image_NPY/NPY_Infrared/ 创建成功
./Image_IMG/IMG_color_depth/ 创建成功
./Image_IMG/IMG_Infrared/ 创建成功


In [138]:
#===检查文件序号值===#
# 定义要创建的目录
Image_path= ["./Image_NPY/NPY_color_depth/", "./Image_NPY/NPY_Infrared/",  
         "./Image_IMG/IMG_color_depth/", "./Image_IMG/IMG_Infrared/"] 

def Check_IMG_SeqNum(root = './', img_suffix = '.npy', img_name_start =['color','depth'], img_name_startnum = 5):
    # root = './'  #指定目录
    Seq_num_Color = []
    Seq_num_Depth = []
    for files in os.listdir(root):
        if (os.path.splitext(files)[1]== img_suffix) & (os.path.splitext(files)[0][:img_name_startnum]==img_name_start[0]):  
            Seq_num_Color.append(int(os.path.splitext(files)[0][-6:]))

        if (os.path.splitext(files)[1]== img_suffix) & (os.path.splitext(files)[0][:img_name_startnum]==img_name_start[1]):  
            Seq_num_Depth.append(int(os.path.splitext(files)[0][-6:]))
    
    if len(Seq_num_Color)==0:     #列表为空  返回0  
        Seq_num_Color.append(0)     
    if len(Seq_num_Depth)==0:     #列表为空  返回0  
        Seq_num_Depth.append(0)   
    if np.equal(Seq_num_Color, Seq_num_Depth).all: 
        print("Check_IMG_SeqNum OK") 
    else:
        print("Warnning: Check_IMG_SeqNum")

    IMGNPY_dict = {'color_npy_num':Seq_num_Color, 'depth_npy_num':Seq_num_Depth}
    return IMGNPY_dict  
#执行=检查文件序号值   
Check_IMG_SeqNum(root=Image_path[0]) 

Check_IMG_SeqNum OK


{'color_npy_num': [0], 'depth_npy_num': [0]}

In [88]:
img_name_start = ['color', 'depth']
print(img_name_start)

['color', 'depth']
